# twitter datasets to graphs

Extract network engagement behavior from twitter datasets: 
- Following
- Likes given
- mentions
- replies 

Build graphs from this


In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import re
import string
from itertools import chain

import networkx as nx
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
# upload all root users
kop_follow = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Taipei/Ko-P/Ko P Following.csv')
sinica_follow = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Taipei/Academia Sinica/Academia Sinica Following.csv')
asus_follow = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Taipei/Asus/Asus Following.csv')
g0v_follow = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Taipei/g0v/g0v Following.csv')
gogoro_follow = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Taipei/Gogoro/Gogoro Following.csv')
tw_news_follow = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Taipei/Taiwan News/Taiwan News Following.csv')

adcash_follow = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tallinn/AdCash/AdCash Following.csv')
bolt_follow = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tallinn/Bolt/Bolt Following.csv')
cleantech_follow = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tallinn/Cleantech ForEst/Cleantech ForEst Following.csv')
postimees_follow = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tallinn/Postimees/Postimees Following.csv')
startup_estonia_follow = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tallinn/Startup Estonia/Startup Estonia Following.csv')
taltech_follow = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tallinn/TalTech/TalTech Following.csv')

checkpoint_follow = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tel Aviv/Check Point Software/Check Point Software Following.csv')
hasadna_follow = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tel Aviv/Hasadna/Hasadna Following.csv')
hayom_follow = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tel Aviv/Israel Hayom/Israel Hayom Following.csv')
moovit_follow = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tel Aviv/Moovit/Moovit Following.csv')
huldai_follow = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tel Aviv/Ron Huldai/Ron Huldai Following.csv')
telavivuni_follow = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tel Aviv/Tel Aviv University/Tel Aviv University Following.csv')

In [3]:
# create a dictionary of datasets and a loop to iterate through it
following = {'kop_follow':kop_follow, 
            'sinica_follow':sinica_follow,
            'asus_follow':asus_follow,
            'g0v_follow':g0v_follow,
            'gogoro_follow':gogoro_follow,
            'tw_news_follow':tw_news_follow,
            'adcash_follow':adcash_follow,
            'bolt_follow':bolt_follow,
            'cleantech_follow':cleantech_follow,
            'postimees_follow':postimees_follow,
            'startup_estonia_follow':startup_estonia_follow,
            'taltech_follow':taltech_follow,
            'checkpoint_follow':checkpoint_follow,
            'hasadna_follow':hasadna_follow,
            'hayom_follow':hayom_follow,
            'moovit_follow':moovit_follow,
            'huldai_follow':huldai_follow,
            'telavivuni_follow':telavivuni_follow
           }
for k in following:
    print(k)

kop_follow
sinica_follow
asus_follow
g0v_follow
gogoro_follow
tw_news_follow
adcash_follow
bolt_follow
cleantech_follow
postimees_follow
startup_estonia_follow
taltech_follow
checkpoint_follow
hasadna_follow
hayom_follow
moovit_follow
huldai_follow
telavivuni_follow


In [4]:
for k, v in following.items():
    v.insert(0,'user', v['query'].str.replace('https://twitter.com/',''))

In [5]:
for k, v in following.items():
    v.insert(1,'follow', v['screenName'])

In [6]:
for k, v in following.items():
    v.insert(2,'like',"")

In [7]:
for k, v in following.items():
    v.insert(3,'reply',"")

In [8]:
for k, v in following.items():
    v.insert(4,'mention',"")

In [9]:
for k, v in following.items():
    v.drop(v.iloc[:,5:], inplace = True, axis = 1)

In [10]:
kop_follow.head()

,user,follow,like,reply,mention
0,KP_Taipei,ogasawara_yoshi,,,
1,KP_Taipei,isaacstonefish,,,
2,KP_Taipei,grzegorzewskif,,,
3,KP_Taipei,DavidColladoM,,,
4,KP_Taipei,suea_thornton,,,


In [11]:
kop_like = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Taipei/Ko-P/Ko P Likes Given.csv')
sinica_like = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Taipei/Academia Sinica/Academia Sinica Likes Given.csv')
asus_like = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Taipei/Asus/Asus Likes Given.csv')
g0v_like = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Taipei/g0v/g0v Likes Given.csv')
gogoro_like = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Taipei/Gogoro/Gogoro Likes Given.csv')
tw_news_like = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Taipei/Taiwan News/Taiwan News Likes Given.csv')

adcash_like = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tallinn/AdCash/AdCash Likes Given.csv')
bolt_like = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tallinn/Bolt/Bolt Likes Given.csv')
cleantech_like = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tallinn/Cleantech ForEst/Cleantech ForEst Likes Given.csv')
postimees_like = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tallinn/Postimees/Postimees Likes Given.csv')
startup_estonia_like = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tallinn/Startup Estonia/Startup Estonia Likes Given.csv')
taltech_like = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tallinn/TalTech/TalTech Likes Given.csv')

checkpoint_like = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tel Aviv/Check Point Software/Check Point Software Likes Given.csv')
hasadna_like = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tel Aviv/Hasadna/Hasadna Likes Given.csv')
hayom_like = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tel Aviv/Israel Hayom/Israel Hayom Likes Given.csv')
moovit_like = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tel Aviv/Moovit/Moovit Likes Given.csv')
huldai_like = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tel Aviv/Ron Huldai/Ron Huldai Likes Given.csv')
telavivuni_like = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tel Aviv/Tel Aviv University/Tel Aviv University Likes Given.csv')

In [12]:
like = {'kop_like':kop_like, 
            'sinica_like':sinica_like,
            'asus_like':asus_like,
            'g0v_like':g0v_like,
            'gogoro_like':gogoro_like,
            'tw_news_like':tw_news_like,
            'adcash_like':adcash_like,
            'bolt_like':bolt_like,
            'cleantech_like':cleantech_like,
            'postimees_like':postimees_like,
            'startup_estonia_like':startup_estonia_like,
            'taltech_like':taltech_like,
            'checkpoint_like':checkpoint_like,
            'hasadna_like':hasadna_like,
            'hayom_like':hayom_like,
            'moovit_like':moovit_like,
            'huldai_like':huldai_like,
            'telavivuni_like':telavivuni_like
           }
for k in like:
    print(k)

kop_like
sinica_like
asus_like
g0v_like
gogoro_like
tw_news_like
adcash_like
bolt_like
cleantech_like
postimees_like
startup_estonia_like
taltech_like
checkpoint_like
hasadna_like
hayom_like
moovit_like
huldai_like
telavivuni_like


In [13]:
for k, v in like.items():
    v.insert(0,'user', v['query'].str.replace('https://twitter.com/',''))

In [14]:
for k, v in like.items():
    v.insert(1,'follow', "")

In [15]:
for k, v in like.items():
    v.insert(2,'like',v['profileUser'].str.replace('https://twitter.com/',''))

In [16]:
for k, v in like.items():
    v.insert(3,'reply',"")

In [17]:
for k, v in like.items():
    v.insert(4,'mention',"")

In [18]:
kop_like.drop(kop_like.iloc[:,5:], inplace = True, axis = 1)

In [19]:
for k, v in like.items():
    v.drop(v.iloc[:,5:], inplace = True, axis = 1)

In [20]:
kop_like.head()

,user,follow,like,reply,mention
0,KP_Taipei,,KP_Taipei,,
1,KP_Taipei,,AusOfficeTPE,,
2,KP_Taipei,,BarackObama,,
3,KP_Taipei,,KP_Taipei,,
4,KP_Taipei,,KP_Taipei,,


In [21]:
kop_tweets = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Taipei/Ko-P/Ko P Tweets & Replies.csv')
sinica_tweets = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Taipei/Academia Sinica/Academia Sinica Tweets & Replies.csv')
asus_tweets = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Taipei/Asus/Asus Tweets & Replies.csv')
g0v_tweets = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Taipei/g0v/g0v Tweets & Replies.csv')
gogoro_tweets = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Taipei/Gogoro/Gogoro Tweets & Replies.csv')
tw_news_tweets = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Taipei/Taiwan News/Taiwan News Tweets & Replies.csv')

adcash_tweets = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tallinn/AdCash/AdCash Tweets & Replies.csv')
bolt_tweets = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tallinn/Bolt/Bolt Tweets & Replies.csv')
cleantech_tweets = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tallinn/Cleantech ForEst/Cleantech ForEst Tweets & Replies.csv')
postimees_tweets = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tallinn/Postimees/Postimees Tweets & Replies.csv')
startup_estonia_tweets = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tallinn/Startup Estonia/Startup Estonia Tweets & Replies.csv')
taltech_tweets = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tallinn/TalTech/TalTech Tweets & Replies.csv')

checkpoint_tweets = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tel Aviv/Check Point Software/Check Point Software Tweets & Replies.csv')
hasadna_tweets = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tel Aviv/Hasadna/Hasadna Tweets & Replies.csv')
hayom_tweets = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tel Aviv/Israel Hayom/Israel Hayom Tweets & Replies.csv')
moovit_tweets = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tel Aviv/Moovit/Moovit Tweets & Replies.csv')
huldai_tweets = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tel Aviv/Ron Huldai/Ron Huldai Tweets & Replies.csv')
telavivuni_tweets = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tel Aviv/Tel Aviv University/Tel Aviv University Tweets & Replies.csv')

In [22]:
tweets = {'kop_tweets':kop_tweets, 
            'sinica_tweets':sinica_tweets,
            'asus_tweets':asus_tweets,
            'g0v_tweets':g0v_tweets,
            'gogoro_tweets':gogoro_tweets,
            'tw_news_tweets':tw_news_tweets,
            'adcash_tweets':adcash_tweets,
            'bolt_tweets':bolt_tweets,
            'cleantech_tweets':cleantech_tweets,
            'postimees_tweets':postimees_tweets,
            'startup_estonia_tweets':startup_estonia_tweets,
            'taltech_tweets':taltech_tweets,
            'checkpoint_tweets':checkpoint_tweets,
            'hasadna_tweets':hasadna_tweets,
            'hayom_tweets':hayom_tweets,
            'moovit_tweets':moovit_tweets,
            'huldai_tweets':huldai_tweets,
            'telavivuni_tweets':telavivuni_tweets
           }
for k in tweets:
    print(k)

kop_tweets
sinica_tweets
asus_tweets
g0v_tweets
gogoro_tweets
tw_news_tweets
adcash_tweets
bolt_tweets
cleantech_tweets
postimees_tweets
startup_estonia_tweets
taltech_tweets
checkpoint_tweets
hasadna_tweets
hayom_tweets
moovit_tweets
huldai_tweets
telavivuni_tweets


In [23]:
for k, v in tweets.items():
    v.insert(0,'user', v['query'].str.replace('https://twitter.com/',''))

In [24]:
for k, v in tweets.items():
    v.insert(1,'follow', "")

In [25]:
for k, v in tweets.items():
    v.insert(2,'like',"")

In [26]:
kop_tweets.insert(3,'reply', kop_tweets['replyTo'].str.replace('@','').fillna(""))
sinica_tweets.insert(3,'reply', sinica_tweets['replyTo'].str.replace('@','').fillna(""))
asus_tweets.insert(3,'reply', asus_tweets['replyTo'].str.replace('@','').fillna(""))
g0v_tweets.insert(3,'reply', g0v_tweets['replyTo'].str.replace('@','').fillna(""))
gogoro_tweets.insert(3,'reply', gogoro_tweets['replyTo'].str.replace('@','').fillna(""))
tw_news_tweets.insert(3,'reply', tw_news_tweets['replyTo'].str.replace('@','').fillna(""))
adcash_tweets.insert(3,'reply', adcash_tweets['replyTo'].str.replace('@','').fillna(""))
bolt_tweets.insert(3,'reply', bolt_tweets['replyTo'].str.replace('@','').fillna(""))
cleantech_tweets.insert(3,'reply', cleantech_tweets['replyTo'].str.replace('@','').fillna(""))
postimees_tweets.insert(3,'reply', "")
startup_estonia_tweets.insert(3,'reply', startup_estonia_tweets['replyTo'].str.replace('@','').fillna(""))
taltech_tweets.insert(3,'reply', taltech_tweets['replyTo'].str.replace('@','').fillna(""))
checkpoint_tweets.insert(3,'reply', checkpoint_tweets['replyTo'].str.replace('@','').fillna(""))
hasadna_tweets.insert(3,'reply', hasadna_tweets['replyTo'].str.replace('@','').fillna(""))
hayom_tweets.insert(3,'reply', hayom_tweets['replyTo'].str.replace('@','').fillna(""))
moovit_tweets.insert(3,'reply', moovit_tweets['replyTo'].str.replace('@','').fillna(""))
huldai_tweets.insert(3,'reply', huldai_tweets['replyTo'].str.replace('@','').fillna(""))
telavivuni_tweets.insert(3,'reply', telavivuni_tweets['replyTo'].str.replace('@','').fillna(""))

In [27]:
kop_tweets.insert(4,'mention', kop_tweets['text'].str.findall(r'(?<![@\w])@(\w{1,25})').apply(','.join))
sinica_tweets.insert(4,'mention', sinica_tweets['text'].str.findall(r'(?<![@\w])@(\w{1,25})').apply(','.join))
asus_tweets.insert(4,'mention', asus_tweets['text'].str.findall(r'(?<![@\w])@(\w{1,25})').apply(','.join))
g0v_tweets.insert(4,'mention', g0v_tweets['text'].str.findall(r'(?<![@\w])@(\w{1,25})').apply(','.join))
gogoro_tweets.insert(4,'mention', gogoro_tweets['text'].str.findall(r'(?<![@\w])@(\w{1,25})').apply(','.join))
tw_news_tweets.insert(4,'mention', tw_news_tweets['text'].str.findall(r'(?<![@\w])@(\w{1,25})').apply(','.join))
adcash_tweets.insert(4,'mention', adcash_tweets['text'].str.findall(r'(?<![@\w])@(\w{1,25})').apply(','.join))
bolt_tweets.insert(4,'mention', bolt_tweets['text'].str.findall(r'(?<![@\w])@(\w{1,25})').apply(','.join))
cleantech_tweets.insert(4,'mention', cleantech_tweets['text'].str.findall(r'(?<![@\w])@(\w{1,25})').apply(','.join))
postimees_tweets.insert(4,'mention', postimees_tweets['text'].str.findall(r'(?<![@\w])@(\w{1,25})').apply(','.join))
startup_estonia_tweets.insert(4,'mention', startup_estonia_tweets['text'].str.findall(r'(?<![@\w])@(\w{1,25})').apply(','.join))
taltech_tweets.insert(4,'mention', taltech_tweets['text'].str.findall(r'(?<![@\w])@(\w{1,25})').apply(','.join))
checkpoint_tweets.insert(4,'mention', checkpoint_tweets['text'].str.findall(r'(?<![@\w])@(\w{1,25})').apply(','.join))
hasadna_tweets.insert(4,'mention', hasadna_tweets['text'].str.findall(r'(?<![@\w])@(\w{1,25})').dropna().apply(','.join))
hayom_tweets.insert(4,'mention', hayom_tweets['text'].str.findall(r'(?<![@\w])@(\w{1,25})').apply(','.join))
moovit_tweets.insert(4,'mention', moovit_tweets['text'].str.findall(r'(?<![@\w])@(\w{1,25})').apply(','.join))
huldai_tweets.insert(4,'mention', huldai_tweets['text'].str.findall(r'(?<![@\w])@(\w{1,25})').apply(','.join))
telavivuni_tweets.insert(4,'mention', telavivuni_tweets['text'].str.findall(r'(?<![@\w])@(\w{1,25})').apply(','.join))

In [28]:
# fix bug null value
hasadna_tweets = hasadna_tweets[hasadna_tweets['mention'].notnull()]

In [29]:
kop_tweets.drop(kop_tweets.iloc[:,5:], inplace = True, axis = 1)

In [30]:
for k, v in tweets.items():
    v.drop(v.iloc[:,5:], inplace = True, axis = 1)

In [31]:
kop_tweets.head()

,user,follow,like,reply,mention
0,KP_Taipei,,,,Twitter
1,KP_Taipei,,,,
2,KP_Taipei,,,,
3,KP_Taipei,,,,
4,KP_Taipei,,,,


In [32]:
# concat all df in the same df
tw_public = pd.concat([kop_follow, kop_like, kop_tweets], axis=0)
tw_corpo = pd.concat([asus_follow, asus_like, asus_tweets], axis=0)
tw_startup = pd.concat([gogoro_follow, gogoro_like, gogoro_tweets], axis=0)
tw_academic = pd.concat([sinica_follow, sinica_like, sinica_tweets], axis=0)
tw_civil = pd.concat([g0v_follow, g0v_like, g0v_tweets], axis=0)
tw_media = pd.concat([tw_news_follow, tw_news_like, tw_news_tweets], axis=0)
il_public = pd.concat([huldai_follow, huldai_like, huldai_tweets], axis=0)
il_corpo = pd.concat([checkpoint_follow, checkpoint_like, checkpoint_tweets], axis=0)
il_startup = pd.concat([moovit_follow, moovit_like, moovit_tweets], axis=0)
il_academic = pd.concat([telavivuni_follow, telavivuni_like, telavivuni_tweets], axis=0)
il_civil = pd.concat([hasadna_follow, hasadna_like, hasadna_tweets], axis=0)
il_media = pd.concat([hayom_follow, hayom_like, hayom_tweets], axis=0)
ee_public = pd.concat([startup_estonia_follow, startup_estonia_like, startup_estonia_tweets], axis=0)
ee_corpo = pd.concat([bolt_follow, bolt_like, bolt_tweets], axis=0)
ee_startup = pd.concat([adcash_follow, adcash_like, adcash_tweets], axis=0)
ee_academic = pd.concat([taltech_follow, taltech_like, taltech_tweets], axis=0)
ee_civil = pd.concat([cleantech_follow, cleantech_like, cleantech_tweets], axis=0)
ee_media = pd.concat([postimees_follow, postimees_like, postimees_tweets], axis=0)

In [33]:
# create a dictionary of datasets and a loop to iterate through it
relation = {'tw_public':tw_public, 
            'tw_corpo':tw_corpo,
            'tw_startup':tw_startup,
            'tw_academic':tw_academic,
            'tw_civil':tw_civil,
            'tw_media':tw_media,
            'il_public':il_public,
            'il_corpo':il_corpo,
            'il_startup':il_startup,
            'il_academic':il_academic,
            'il_civil':il_civil,
            'il_media':il_media,
            'ee_public':ee_public,
            'ee_corpo':ee_corpo,
            'ee_startup':ee_startup,
            'ee_academic':ee_academic,
            'ee_civil':ee_civil,
            'ee_media':ee_media
           }
for k in relation:
    print(k)

tw_public
tw_corpo
tw_startup
tw_academic
tw_civil
tw_media
il_public
il_corpo
il_startup
il_academic
il_civil
il_media
ee_public
ee_corpo
ee_startup
ee_academic
ee_civil
ee_media


In [34]:
for k, v in relation.items():
    v['target'] = v['follow'] + "," + v['like'] + "," + v['reply'] + "," + v['mention']

In [35]:
# correct one bug, should correct root later
il_civil = il_civil[il_civil['target'].notnull()]

In [36]:
# return list from series of comma-separated strings
def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

# calculate lengths of splits
lens_tw_public = tw_public['target'].str.split(',').map(len)
lens_tw_corpo = tw_corpo['target'].str.split(',').map(len)
lens_tw_startup = tw_startup['target'].str.split(',').map(len)
lens_tw_academic = tw_academic['target'].str.split(',').map(len)
lens_tw_civil = tw_civil['target'].str.split(',').map(len)
lens_tw_media = tw_media['target'].str.split(',').map(len)
lens_il_public = il_public['target'].str.split(',').map(len)
lens_il_corpo = il_corpo['target'].str.split(',').map(len)
lens_il_startup = il_startup['target'].str.split(',').map(len)
lens_il_academic = il_academic['target'].str.split(',').map(len)
lens_il_civil = il_civil['target'].str.split(',').map(len)
lens_il_media = il_media['target'].str.split(',').map(len)
lens_ee_public = ee_public['target'].str.split(',').map(len)
lens_ee_corpo = ee_corpo['target'].str.split(',').map(len)
lens_ee_startup = ee_startup['target'].str.split(',').map(len)
lens_ee_academic = ee_academic['target'].str.split(',').map(len)
lens_ee_civil = ee_civil['target'].str.split(',').map(len)
lens_ee_media = ee_media['target'].str.split(',').map(len)

# create new dataframe, repeating or chaining as appropriate
tw_public = pd.DataFrame({'user': np.repeat(tw_public['user'], lens_tw_public),
                          'target': chainer(tw_public['target'])})
tw_corpo = pd.DataFrame({'user': np.repeat(tw_corpo['user'], lens_tw_corpo),
                          'target': chainer(tw_corpo['target'])})
tw_startup = pd.DataFrame({'user': np.repeat(tw_startup['user'], lens_tw_startup),
                          'target': chainer(tw_startup['target'])})
tw_academic = pd.DataFrame({'user': np.repeat(tw_academic['user'], lens_tw_academic),
                          'target': chainer(tw_academic['target'])})
tw_civil = pd.DataFrame({'user': np.repeat(tw_civil['user'], lens_tw_civil),
                          'target': chainer(tw_civil['target'])})
tw_media = pd.DataFrame({'user': np.repeat(tw_media['user'], lens_tw_media),
                          'target': chainer(tw_media['target'])})
il_public = pd.DataFrame({'user': np.repeat(il_public['user'], lens_il_public),
                          'target': chainer(il_public['target'])})
il_corpo = pd.DataFrame({'user': np.repeat(il_corpo['user'], lens_il_corpo),
                          'target': chainer(il_corpo['target'])})
il_startup = pd.DataFrame({'user': np.repeat(il_startup['user'], lens_il_startup),
                          'target': chainer(il_startup['target'])})
il_academic = pd.DataFrame({'user': np.repeat(il_academic['user'], lens_il_academic),
                          'target': chainer(il_academic['target'])})
il_civil = pd.DataFrame({'user': np.repeat(il_civil['user'], lens_il_civil),
                          'target': chainer(il_civil['target'])})
il_media = pd.DataFrame({'user': np.repeat(il_media['user'], lens_il_media),
                          'target': chainer(il_media['target'])})
ee_public = pd.DataFrame({'user': np.repeat(ee_public['user'], lens_ee_public),
                          'target': chainer(ee_public['target'])})
ee_corpo = pd.DataFrame({'user': np.repeat(ee_corpo['user'], lens_ee_corpo),
                          'target': chainer(ee_corpo['target'])})
ee_startup = pd.DataFrame({'user': np.repeat(ee_startup['user'], lens_ee_startup),
                          'target': chainer(ee_startup['target'])})
ee_academic = pd.DataFrame({'user': np.repeat(ee_academic['user'], lens_ee_academic),
                          'target': chainer(ee_academic['target'])})
ee_civil = pd.DataFrame({'user': np.repeat(ee_civil['user'], lens_ee_civil),
                          'target': chainer(ee_civil['target'])})
ee_media = pd.DataFrame({'user': np.repeat(ee_media['user'], lens_ee_media),
                          'target': chainer(ee_media['target'])})

In [37]:
# replace empty strings in target column by NaN
tw_public['target'].replace('', np.nan, inplace=True)
tw_corpo['target'].replace('', np.nan, inplace=True)
tw_startup['target'].replace('', np.nan, inplace=True)
tw_academic['target'].replace('', np.nan, inplace=True)
tw_civil['target'].replace('', np.nan, inplace=True)
tw_media['target'].replace('', np.nan, inplace=True)
il_public['target'].replace('', np.nan, inplace=True)
il_corpo['target'].replace('', np.nan, inplace=True)
il_startup['target'].replace('', np.nan, inplace=True)
il_academic['target'].replace('', np.nan, inplace=True)
il_civil['target'].replace('', np.nan, inplace=True)
il_media['target'].replace('', np.nan, inplace=True)
ee_public['target'].replace('', np.nan, inplace=True)
ee_corpo['target'].replace('', np.nan, inplace=True)
ee_startup['target'].replace('', np.nan, inplace=True)
ee_academic['target'].replace('', np.nan, inplace=True)
ee_civil['target'].replace('', np.nan, inplace=True)
ee_media['target'].replace('', np.nan, inplace=True)

In [38]:
# drop null values
tw_public.dropna(subset=['target'], inplace=True)
tw_corpo.dropna(subset=['target'], inplace=True)
tw_startup.dropna(subset=['target'], inplace=True)
tw_academic.dropna(subset=['target'], inplace=True)
tw_civil.dropna(subset=['target'], inplace=True)
tw_media.dropna(subset=['target'], inplace=True)
il_public.dropna(subset=['target'], inplace=True)
il_corpo.dropna(subset=['target'], inplace=True)
il_startup.dropna(subset=['target'], inplace=True)
il_academic.dropna(subset=['target'], inplace=True)
il_civil.dropna(subset=['target'], inplace=True)
il_media.dropna(subset=['target'], inplace=True)
ee_public.dropna(subset=['target'], inplace=True)
ee_corpo.dropna(subset=['target'], inplace=True)
ee_startup.dropna(subset=['target'], inplace=True)
ee_academic.dropna(subset=['target'], inplace=True)
ee_civil.dropna(subset=['target'], inplace=True)
ee_media.dropna(subset=['target'], inplace=True)

In [39]:
# rebuild url from handle
tw_public['url'] = tw_public['target'].apply(lambda x: 'https://twitter.com/' + x)
tw_corpo['url'] = tw_corpo['target'].apply(lambda x: 'https://twitter.com/' + x)
tw_startup['url'] = tw_startup['target'].apply(lambda x: 'https://twitter.com/' + x)
tw_academic['url'] = tw_academic['target'].apply(lambda x: 'https://twitter.com/' + x)
tw_civil['url'] = tw_civil['target'].apply(lambda x: 'https://twitter.com/' + x)
tw_media['url'] = tw_media['target'].apply(lambda x: 'https://twitter.com/' + x)
il_public['url'] = il_public['target'].apply(lambda x: 'https://twitter.com/' + x)
il_corpo['url'] = il_corpo['target'].apply(lambda x: 'https://twitter.com/' + x)
il_startup['url'] = il_startup['target'].apply(lambda x: 'https://twitter.com/' + x)
il_academic['url'] = il_academic['target'].apply(lambda x: 'https://twitter.com/' + x)
il_civil['url'] = il_civil['target'].apply(lambda x: 'https://twitter.com/' + x)
il_media['url'] = il_media['target'].apply(lambda x: 'https://twitter.com/' + x)
ee_public['url'] = ee_public['target'].apply(lambda x: 'https://twitter.com/' + x)
ee_corpo['url'] = ee_corpo['target'].apply(lambda x: 'https://twitter.com/' + x)
ee_startup['url'] = ee_startup['target'].apply(lambda x: 'https://twitter.com/' + x)
ee_academic['url'] = ee_academic['target'].apply(lambda x: 'https://twitter.com/' + x)
ee_civil['url'] = ee_civil['target'].apply(lambda x: 'https://twitter.com/' + x)
ee_media['url'] = ee_media['target'].apply(lambda x: 'https://twitter.com/' + x)

In [40]:
# count occurences
tw_public['freq'] = tw_public.groupby('target')['target'].transform('count')
tw_corpo['freq'] = tw_corpo.groupby('target')['target'].transform('count')
tw_startup['freq'] = tw_startup.groupby('target')['target'].transform('count')
tw_academic['freq'] = tw_academic.groupby('target')['target'].transform('count')
tw_civil['freq'] = tw_civil.groupby('target')['target'].transform('count')
tw_media['freq'] = tw_media.groupby('target')['target'].transform('count')
il_public['freq'] = il_public.groupby('target')['target'].transform('count')
il_corpo['freq'] = il_corpo.groupby('target')['target'].transform('count')
il_startup['freq'] = il_startup.groupby('target')['target'].transform('count')
il_academic['freq'] = il_academic.groupby('target')['target'].transform('count')
il_civil['freq'] = il_civil.groupby('target')['target'].transform('count')
il_media['freq'] = il_media.groupby('target')['target'].transform('count')
ee_public['freq'] = ee_public.groupby('target')['target'].transform('count')
ee_corpo['freq'] = ee_corpo.groupby('target')['target'].transform('count')
ee_startup['freq'] = ee_startup.groupby('target')['target'].transform('count')
ee_academic['freq'] = ee_academic.groupby('target')['target'].transform('count')
ee_civil['freq'] = ee_civil.groupby('target')['target'].transform('count')
ee_media['freq'] = ee_media.groupby('target')['target'].transform('count')

In [41]:
tw_public = tw_public.drop_duplicates()
tw_corpo = tw_corpo.drop_duplicates()
tw_startup = tw_startup.drop_duplicates()
tw_academic = tw_academic.drop_duplicates()
tw_civil = tw_civil.drop_duplicates()
tw_media = tw_media.drop_duplicates()
il_public = il_public.drop_duplicates()
il_corpo = il_corpo.drop_duplicates()
il_startup = il_startup.drop_duplicates()
il_academic = il_academic.drop_duplicates()
il_civil = il_civil.drop_duplicates()
il_media = il_media.drop_duplicates()
ee_public = ee_public.drop_duplicates()
ee_corpo = ee_corpo.drop_duplicates()
ee_startup = ee_startup.drop_duplicates()
ee_academic = ee_academic.drop_duplicates()
ee_civil = ee_civil.drop_duplicates()
ee_media = ee_media.drop_duplicates()

In [43]:
taipei = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Taipei/#Taipei.csv')
tallinn = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tallinn/#Tallinn.csv')
telaviv = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tel Aviv/#Telaviv.csv')

In [44]:
city = {'taipei':taipei,
       'tallinn':tallinn,
       'telaviv':telaviv} 

for k in city:
    print(k)

taipei
tallinn
telaviv


In [45]:
for k, v in city.items():
    v.insert(0,'user', v['twitterProfile'].str.replace('https://twitter.com/',''))

In [46]:
for k, v in city.items():
    v.insert(1,'target', v['content'].str.findall(r'(?<![@\w])@(\w{1,25})').apply(','.join))

In [47]:
for k, v in city.items():
    v.drop(v.iloc[:,2:], inplace = True, axis = 1)

In [48]:
def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

lens_taipei = taipei['target'].str.split(',').map(len)
lens_telaviv = telaviv['target'].str.split(',').map(len)
lens_tallinn = tallinn['target'].str.split(',').map(len)

taipei = pd.DataFrame({'user': np.repeat(taipei['user'], lens_taipei),
                          'target': chainer(taipei['target'])})
telaviv = pd.DataFrame({'user': np.repeat(telaviv['user'], lens_telaviv),
                          'target': chainer(telaviv['target'])})
tallinn = pd.DataFrame({'user': np.repeat(tallinn['user'], lens_tallinn),
                          'target': chainer(tallinn['target'])})

In [49]:
taipei['target'].replace('', np.nan, inplace=True)
telaviv['target'].replace('', np.nan, inplace=True)
tallinn['target'].replace('', np.nan, inplace=True)

In [50]:
taipei.dropna(subset=['target'], inplace=True)
telaviv.dropna(subset=['target'], inplace=True)
tallinn.dropna(subset=['target'], inplace=True)

In [51]:
taipei['url'] = taipei['target'].apply(lambda x: 'https://twitter.com/' + x)
telaviv['url'] = telaviv['target'].apply(lambda x: 'https://twitter.com/' + x)
tallinn['url'] = tallinn['target'].apply(lambda x: 'https://twitter.com/' + x)

In [52]:
taipei['freq'] = taipei.groupby('target')['target'].transform('count')
telaviv['freq'] = telaviv.groupby('target')['target'].transform('count')
tallinn['freq'] = tallinn.groupby('target')['target'].transform('count')

In [53]:
taipei = taipei.drop_duplicates()
telaviv = telaviv.drop_duplicates()
tallinn = tallinn.drop_duplicates()

In [54]:
# merge files by city_id
_taipei = pd.concat([taipei,
                     tw_public,
                     tw_corpo,
                     tw_startup,
                     tw_academic,
                     tw_civil,
                     tw_media
                     ], axis=0)

_telaviv = pd.concat([telaviv,
                      il_public,
                      il_corpo,
                      il_startup,
                      il_academic,
                      il_civil,
                      il_media
                     ], axis=0)

_tallinn = pd.concat([tallinn,
                      ee_public,
                      ee_corpo,
                      ee_startup,
                      ee_academic,
                      ee_civil,
                      ee_media
                     ], axis=0)

In [ ]:
# merge files by stakeholders' category
public = pd.concat([tw_public,
                   il_public,
                   ee_public], axis=0)
corpo = pd.concat([tw_corpo,
                   il_corpo,
                   ee_corpo], axis=0)
startup = pd.concat([tw_startup,
                   il_startup,
                   ee_startup], axis=0)
academic = pd.concat([tw_academic,
                   il_academic,
                   ee_academic], axis=0)
civil = pd.concat([tw_civil,
                   il_civil,
                   ee_civil], axis=0)
media = pd.concat([tw_media,
                   il_media,
                   ee_media], axis=0)

In [55]:
casestudies = pd.concat([_taipei, _telaviv, _tallinn], axis=0)

In [ ]:
# create graphs
Graphtype = nx.Graph()
G_tp = nx.from_pandas_edgelist(_taipei, source='user', target='target', edge_attr='freq', create_using=Graphtype)
nx.write_graphml(G_tp,"taipei_graph.graphml")

In [ ]:
G_ta = nx.from_pandas_edgelist(_telaviv, source='user', target='target', edge_attr='freq', create_using=Graphtype)
nx.write_graphml(G_ta,"telaviv_graph.graphml")

In [ ]:
G_tl = nx.from_pandas_edgelist(_tallinn, source='user', target='target', edge_attr='freq', create_using=Graphtype)
nx.write_graphml(G_tl,"tallinn_graph.graphml")

In [ ]:
G_3 = nx.from_pandas_edgelist(casestudies, source='user', target='target', edge_attr='freq', create_using=Graphtype)
nx.write_graphml(G_3,"casestudies_graph.graphml")

In [ ]:
G_public = nx.from_pandas_edgelist(public, source='user', target='target', edge_attr='freq', create_using=Graphtype)
nx.write_graphml(G_public,"public_graph.graphml")

In [ ]:
G_corpo = nx.from_pandas_edgelist(corpo, source='user', target='target', edge_attr='freq', create_using=Graphtype)
nx.write_graphml(G_corpo,"corpo_graph.graphml")

In [ ]:
G_startup = nx.from_pandas_edgelist(startup, source='user', target='target', edge_attr='freq', create_using=Graphtype)
nx.write_graphml(G_startup,"startup_graph.graphml")

In [ ]:
G_academic = nx.from_pandas_edgelist(academic, source='user', target='target', edge_attr='freq', create_using=Graphtype)
nx.write_graphml(G_academic,"academic_graph.graphml")

In [ ]:
G_civil = nx.from_pandas_edgelist(civil, source='user', target='target', edge_attr='freq', create_using=Graphtype)
nx.write_graphml(G_civil,"civil_graph.graphml")

In [ ]:
G_media = nx.from_pandas_edgelist(media, source='user', target='target', edge_attr='freq', create_using=Graphtype)
nx.write_graphml(G_media,"media_graph.graphml")